In [128]:
import requests
import pandas as pd

url = "https://api.binance.com/api/v3/depth"

params = {
    "symbol":"BTCUSDT",
    "limit":5000,
}

In [129]:
data = requests.get(url, params=params).json()
data

{'lastUpdateId': 26517804394,
 'bids': [['20857.98000000', '0.00239000'],
  ['20857.87000000', '0.08415000'],
  ['20857.86000000', '0.03700000'],
  ['20857.84000000', '0.00575000'],
  ['20857.81000000', '0.04910000'],
  ['20857.78000000', '0.00239000'],
  ['20857.77000000', '0.00519000'],
  ['20857.75000000', '0.04858000'],
  ['20857.62000000', '0.00060000'],
  ['20857.58000000', '0.04698000'],
  ['20857.57000000', '0.08415000'],
  ['20857.55000000', '0.04407000'],
  ['20857.23000000', '0.02815000'],
  ['20857.22000000', '0.00540000'],
  ['20857.20000000', '0.00575000'],
  ['20857.18000000', '0.01973000'],
  ['20857.16000000', '0.03554000'],
  ['20857.14000000', '0.00062000'],
  ['20857.13000000', '0.20767000'],
  ['20857.11000000', '0.02092000'],
  ['20857.10000000', '0.00810000'],
  ['20857.09000000', '2.35271000'],
  ['20857.08000000', '0.00500000'],
  ['20857.04000000', '0.02576000'],
  ['20857.00000000', '0.03013000'],
  ['20856.97000000', '0.27132000'],
  ['20856.96000000', '0.95

In [130]:
bid_levels = pd.DataFrame(data["bids"], columns = ["price","quantity"], dtype=float)
bid_levels["side"] = "bid"
bid_levels

,price,quantity,side
0,20857.98,0.00239,bid
1,20857.87,0.08415,bid
2,20857.86,0.03700,bid
3,20857.84,0.00575,bid
4,20857.81,0.04910,bid
...,...,...,...
4995,20559.03,0.00164,bid
4996,20559.02,0.00066,bid
4997,20559.00,0.02832,bid
4998,20558.97,0.00062,bid


In [131]:
ask_levels = pd.DataFrame(data["asks"], columns = ["price","quantity"], dtype=float)
ask_levels["side"] = "ask"
ask_levels

,price,quantity,side
0,20858.90,0.20840,ask
1,20858.91,0.00478,ask
2,20858.93,0.00957,ask
3,20858.94,0.19314,ask
4,20859.00,0.03249,ask
...,...,...,...
4995,21140.18,0.00051,ask
4996,21140.19,0.00118,ask
4997,21140.26,0.00139,ask
4998,21140.36,0.00481,ask


In [132]:
print(min(ask_levels.price))

20858.9


In [123]:
print(pd.concat([bid_levels, ask_levels]).sort_values("price", ascending=False).to_string())

         price   quantity side
4999  21133.41    0.00204  ask
4998  21133.36    0.00384  ask
4997  21133.33    0.05365  ask
4996  21133.26    0.69417  ask
4995  21133.21    0.00193  ask
4994  21133.16    0.00712  ask
4993  21133.00    0.32274  ask
4992  21132.93    0.00071  ask
4991  21132.91    0.02304  ask
4990  21132.90    0.00220  ask
4989  21132.75    0.01586  ask
4988  21132.74    0.00053  ask
4987  21132.65    0.00062  ask
4986  21132.64    0.00616  ask
4985  21132.50    0.00057  ask
4984  21132.49    0.00049  ask
4983  21132.35    0.00508  ask
4982  21132.32    0.00059  ask
4981  21132.31    0.00095  ask
4980  21132.23    0.00337  ask
4979  21132.20    0.00294  ask
4978  21132.17    0.00058  ask
4977  21132.09    0.00056  ask
4976  21132.00    0.08788  ask
4975  21131.90    0.00067  ask
4974  21131.87    0.00166  ask
4973  21131.86    0.00732  ask
4972  21131.70    0.00073  ask
4971  21131.64    0.00074  ask
4970  21131.63    0.00073  ask
4969  21131.57    0.01500  ask
4968  21

In [139]:
threshold = 0.01

print(max(bid_levels.price))
print(min(ask_levels.price))

mid_price = (max(bid_levels.price) + min(ask_levels.price))/2
mid_price

20857.98
20858.9


20858.440000000002

In [140]:
assert max(ask_levels.price) > (1+threshold)*mid_price, "insufficient market depth"
assert min(bid_levels.price) < (1-threshold)*mid_price, "insufficient market depth"

In [141]:
print("Ask threshold:", (1+threshold)*mid_price)
print("Mid Price:", mid_price)
print("Bid threshold:", (1-threshold)*mid_price)

Ask threshold: 21067.024400000002
Mid Price: 20858.440000000002
Bid threshold: 20649.855600000003


In [142]:
print("Max Ask:", max(ask_levels.price))
print("Min Bid:", min(bid_levels.price))

Max Ask: 21140.42
Min Bid: 20558.95


In [143]:
ask_vol =  ask_levels.quantity[ ask_levels.price < (1+threshold)*mid_price].sum()
bid_vol =  bid_levels.quantity[ bid_levels.price > (1-threshold)*mid_price].sum()

In [144]:
print("Ask Volume:", ask_vol)
print("Bid Volume:", bid_vol)

Ask Volume: 1463.06151
Bid Volume: 1085.78395


In [145]:
print("Orderbook Imbalance:", bid_vol - ask_vol)

Orderbook Imbalance: -377.27756
